In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
from tqdm.auto import tqdm

from santa.metrics import PerplexityCalculator
from santa.utils import load_logs, save_logs, save_text


scorer = PerplexityCalculator(model_path="google/gemma-2-9b")

/opt/mamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100% 8/8 [00:09<00:00,  1.15s/it]


In [2]:
history = {}
bs = 8

In [1]:
def insert_swap_search(text, window_size, scorer, history, margin=0.0, bs=8):
    best_text = text
    best_score = scorer.get_perplexity(text, batch_size=bs)
    updated_texts, updated_scores = [], []
    if best_text not in history:
        history[best_text] = best_score
    tokens = text.split()
    for i in tqdm(range(len(tokens))):
        new_texts = []
        # insert探索
        main_tokens = tokens[:i] + tokens[i+window_size:]
        sub_tokens = tokens[i: i+window_size]
        for j in range(len(main_tokens)):
            new_tokens = main_tokens[:j] + sub_tokens + main_tokens[j:]
            assert len(new_tokens) == 100 and len(set(new_tokens) - set(tokens)) == 0 and len(set(tokens) - set(new_tokens)) == 0
            new_text = " ".join(new_tokens)
            if new_text not in history:
                new_texts.append(new_text)
        # swap探索
        for j in range(i+1, len(tokens)):
            tokens[i], tokens[j] = tokens[j], tokens[i]
            new_text = " ".join(tokens)
            tokens[i], tokens[j] = tokens[j], tokens[i]  # もとに戻す
            if new_text not in history and new_text not in new_texts:
                new_texts.append(new_text)
        scores = scorer.get_perplexity(new_texts, batch_size=bs)
        for new_text, new_score in zip(new_texts, scores, strict=True):
            history[new_text] = new_score
            if new_score < best_score + margin:
                updated_texts.append(new_text)
                updated_scores.append(new_score)
                print(f"{new_score:.5f}")
                print(new_text)
    return updated_texts, updated_scores, history

In [6]:
target_id = 5
candidate_texts = [
    "from and and the and as is in it of not that the to we with you advent card carol cheer chocolate chimney drive eat family fireplace game gifts give greeting have holiday hope joy laugh magi merry night of the night peace puzzle relax season sing sleep star visit walk wish workshop angel bake beard believe bow candy candle cheer chimney cookie decorations doll dream elf eggnog fireplace fruitcake gingerbread grinch holly hohoho jingle jump kaggle milk mistletoe naughty nice nutcracker ornament ornament peppermint polar poinsettia reindeer scrooge sleigh snowglobe stocking toy unwrap wrapping paper wonder workshop wreath yuletide",
]
candidate_scores = [
    scorer.get_perplexity(text)
    for text in candidate_texts
]
candidate_scores

[31.726850265228947]

In [ ]:
margin = 0.0
iteration = 0
min_window_size = 1
max_window_size = 4
while len(candidate_texts):
    iteration += 1
    text = candidate_texts.pop(0)
    score = candidate_scores.pop(0)
    print(f"[iteration {iteration:>03}] score: {score:.5f}")
    updated_texts, updated_scores = [], []
    for window_size in range(min_window_size, max_window_size):
        tmp_updated_texts, tmp_updated_scores, history = insert_swap_search(text, window_size, scorer, history, margin=margin, bs=bs)
        updated_texts += tmp_updated_texts
        updated_scores += tmp_updated_scores
    for new_text, new_score in zip(updated_texts, updated_scores):
        save_text(new_text, new_score, target_id, output_dir="./output/")
    candidate_texts += updated_texts
    candidate_scores += updated_scores
    candidate_texts = np.array(candidate_texts)
    candidate_scores = np.array(candidate_scores)
    indices = candidate_scores.argsort()
    candidate_texts = candidate_texts[indices].tolist()
    candidate_scores = candidate_scores[indices].tolist()

In [ ]:
sorted(history.items(), key=lambda x: x[1])[:10]